![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_in_Spark_NLP_RoBertaSentenceEmbeddings.ipynb)

## Import RoBertaSentenceEmbeddings models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 3.1.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for RoBertaSentenceEmbeddings from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use RoBertaSentenceEmbeddings models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.39.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.


In [ ]:
!pip install -q transformers==4.39.3 tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 19.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 3.19.6 which is incompatible.
pandas

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [FacebookAI/roberta-base](https://huggingface.co/FacebookAI/roberta-base) model from HuggingFace as an example
- In addition to `TFRobertaModel` we also need to save the `RobertaTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.
- Since `FacebookAI/roberta-base` model is PyTorch we will use `from_pt=True` param to convert it to TensorFlow

In [ ]:
from transformers import RobertaTokenizer, TFRobertaModel
import tensorflow as tf

# FacebookAI/roberta-base
MODEL_NAME = 'FacebookAI/roberta-base'

roberta_tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME, return_tensors="pt")

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFRobertaModel.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFRobertaModel.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

roberta_tokenizer.save_pretrained("./{}_tokenizer".format(MODEL_NAME))
model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

try downloading TF weights


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.embeddings.position_ids', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 487172
-rw-r--r-- 1 root root       649 Jun 12 14:21 config.json
drwxr-xr-x 3 root root      4096 Jun 12 14:21 saved_model
-rw-r--r-- 1 root root 498849472 Jun 12 14:21 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 8312
drwxr-xr-x 2 root root    4096 Jun 12 14:21 assets
-rw-r--r-- 1 root root      56 Jun 12 14:21 fingerprint.pb
-rw-r--r-- 1 root root  153179 Jun 12 14:21 keras_metadata.pb
-rw-r--r-- 1 root root 8339458 Jun 12 14:21 saved_model.pb
drwxr-xr-x 2 root root    4096 Jun 12 14:21 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 1432
-rw-r--r-- 1 root root 456318 Jun 12 14:20 merges.txt
-rw-r--r-- 1 root root    958 Jun 12 14:20 special_tokens_map.json
-rw-r--r-- 1 root root   1217 Jun 12 14:20 tokenizer_config.json
-rw-r--r-- 1 root root 999355 Jun 12 14:20 vocab.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `vocab.txt` and `merges.txt` files from the tokenizer
- all we need is to copy `sentencepiece.bpe.model` and `merges.txt` files into `saved_model/1/assets` which Spark NLP will look for

In [ ]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's save the vocab as txt file
with open('{}_tokenizer/vocab.txt'.format(MODEL_NAME), 'w') as f:
    for item in roberta_tokenizer.get_vocab().keys():
        f.write("%s\n" % item)

!cp {MODEL_NAME}_tokenizer/vocab.txt {MODEL_NAME}/saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/merges.txt {MODEL_NAME}/saved_model/1/assets

## Import and Save RoBertaSentenceEmbeddings in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.6 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


- Let's use `loadSavedModel` functon in `RoBertaSentenceEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `RoBertaSentenceEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [ ]:
from sparknlp.annotator import *

MODEL_NAME = 'FacebookAI/roberta-base'

# All these params should be identical to the original ONNX model
roberta = RoBertaSentenceEmbeddings.loadSavedModel(f"{MODEL_NAME}/saved_model/1", spark)\
    .setInputCols(["sentence"])\
    .setOutputCol("roberta")\
    .setCaseSensitive(True)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
roberta.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX RoBertaSentenceEmbeddings model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 494980
drwxr-xr-x 5 root root      4096 Jun 12 14:24 fields
drwxr-xr-x 2 root root      4096 Jun 12 14:24 metadata
-rw-r--r-- 1 root root 506845163 Jun 12 14:24 roberta_tensorflow


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaSentenceEmbeddings model 😊

In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

roberta_loaded = RoBertaSentenceEmbeddings.load(f"{MODEL_NAME}_spark_nlp")\
    .setInputCols(["sentence"])\
    .setOutputCol("roberta")

pipeline = Pipeline(
    stages = [
        document_assembler,
        sentencerDL,
        roberta_loaded
  ])

data = spark.createDataFrame([['William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor,and philanthropist.']]).toDF("text")
model = pipeline.fit(data)
result = model.transform(data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]


In [ ]:
result.selectExpr("explode(roberta.embeddings[0]) as embeddings").show()

+------------+
|  embeddings|
+------------+
| -0.10308697|
| -0.14017585|
|-0.022629127|
| -0.22931732|
|  0.09049399|
|  0.09779589|
| -0.18998949|
|  0.20610748|
|  0.16861194|
| -0.33205318|
|  0.04665165|
| -0.08459879|
|0.0036535591|
|  0.05269509|
|  0.10895155|
| -0.31052336|
|  0.09852609|
|  0.34158343|
|  -0.2502671|
|   0.3526209|
+------------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of RoBertaSentenceEmbeddings models from HuggingFace 🤗 in Spark NLP 🚀
